# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

1. The appropriate test is a two sample test for a difference in proportions. The central limit theoreom does not apply because we only have one sample. If we were to take repeated samples of the same size, the CLT would apply.
2. The null hypothesis is that the proportion of call backs for black sounding names = proportion of call backs for white sounding names. The alternative hypothesis is that the proportions are NOT equal.

In [4]:
w = data[data.race=='w']
b = data[data.race=='b']
print(np.mean(w['call']) - np.mean(b['call']),np.mean(w['call']),np.mean(b['call']))

0.032032855 0.09650924 0.064476386


In [56]:
# Your solution to Q3 here
obs_diff = np.mean(w['call']) - np.mean(b['call'])
t,p = stats.ttest_ind(w['call'],b['call'])
print ('Independent ttest results: TStatistic = ', t,'p-value = ', p)

data_permuted = data[['race','call']]
prop_diff_permuted = np.empty(100)
for i in range(100):
    W = data_permuted[data_permuted.race=='w']
    B = data_permuted[data_permuted.race=='b']
    data_permuted['call'] = np.random.permutation(data_permuted['call'])
    prop_diff_permuted[i] = np.mean(W['call']) - np.mean(B['call'])
p = np.sum(prop_diff_permuted>=obs_diff)/len(prop_diff_permuted)
print('Permuted replicates p-value:',p)

replicates = np.empty(100)
for i in range (100):
    replicates[i] = np.mean(np.random.choice(w['call'],500)) - np.mean(np.random.choice(b['call'],500))
lower,upper = np.percentile(replicates,[2.5,97.5])
p = np.sum(replicates<= 0)/len(replicates)
print('95% confidence interval:', lower,'-', upper)
print('Replicates p-value: ', p)

Independent ttest results: TStatistic =  4.114705290861751 p-value =  3.940802103128886e-05


C:\Users\nhcam\Miniconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Permuted replicates p-value: 0.01
95% confidence interval: 0.0039000008255243306 - 0.06704999785870311
Replicates p-value:  0.01


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

4. Racial discrimination still appears to play a role in hiring decisions across the country. Resumes with white sounding names were called back at a rate of 9.7%, while resumes with black sounding names were called back 6.4% of the time. While only a difference of 3.3%, this result is statistically significant. Running a 2 sample t-test for the difference of mean call back rate between white names and black names gave us a p-value < 0.001. This means that if there wasn't a true difference in the population we would only see a difference this extreme in 1 out of every 1000 or more samples of 235 resumes. We can confidently say that identical resumes with racially implied names have different call back rates.

5. No, we cannot say that race/name is the most important factor in callback success without examining whether other variables correlate positively (or negatively I suppose) with callback success. Below I listed all the numerical variables and their correlation to callback in descending order. The strongest correlation is with special skills, and even that only has a pearson r of 0.11. Even though none of these correlations are particularly strong, it would still be inappropriate to say that race has the biggest effect on call back rate without first putting together a full model and examining the coefficients. At this point all we can really say is that race DOES play a role resume call back rate when that shouldn't be the case.

In [10]:
numerical = data.dtypes[data.dtypes != 'object'].index
numerical_data = data[numerical]
#f, ax = plt.subplots(figsize=(12, 10))
#plt.title('Pearson Correlation of Callback Success')
#sns.heatmap(numerical_data.corr(),linewidths=0.25,vmax=1.0, square=True, cmap="YlGnBu", linecolor='black', annot=True)
numerical_data.corr()['call'].sort_values(ascending=False)

call                  1.000000
specialskills         0.111074
honors                0.071951
empholes              0.071888
adid                  0.063178
yearsexp              0.061436
linc                  0.049649
offsupport            0.047783
lmedhhinc             0.047699
fraccolp              0.047016
occupspecific         0.040548
parent_emp            0.039060
othservice            0.036114
transcom              0.035558
fracwhite             0.035148
occupbroad            0.034536
fraccolp_empzip       0.026118
email                 0.025880
h                     0.025835
fed                   0.014471
fracdropout_empzip    0.013951
fracblack_empzip      0.009882
parent_sales          0.008430
volunteer             0.007197
linc_empzip           0.006883
busservice            0.006882
secretary             0.004038
eoe                   0.003092
comreq                0.002421
retailsales           0.002336
ofjobs                0.002311
education            -0.005748
missind 